In [56]:
import os
import sys

# Init project path
PROJECT_DIR = os.getcwd() + "/../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import os
import re
import pandas as pd
import psycopg2


table_name = "train"
col_names = ["sex"]

col_names_snake_case = [
    re.sub("([A-Z])", lambda x: "_" + x.group(1).lower(), col_name).lstrip("_")
    for col_name in col_names
]

with psycopg2.connect(
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"],
    port=5432,
    database=os.environ["PROJECT_NAME"],
) as conn:
    df = pd.read_sql(
        "SELECT index, {} FROM {};".format(", ".join(col_names_snake_case), table_name),
        con=conn,
    ).set_index("index")

In [60]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://{}:{}@{}:5432/{}".format(
        os.environ["POSTGRES_USER"],
        os.environ["POSTGRES_PASSWORD"],
        os.environ["POSTGRES_HOST"],
        os.environ["PROJECT_NAME"],
    )
)
df.to_sql(name=table_name+"_tmp", con=engine, if_exists="replace", index=True)

dtype_mapper = {
    np.dtype("object"): "TEXT",
    np.dtype("int64"): "BIGINT",
    
}

queries = [
    """
    ALTER TABLE {0} DROP COLUMN IF EXISTS {1};
    ALTER TABLE {0} ADD COLUMN {1} {2};
    UPDATE {0} t1
    SET    {1} = t2.{1}
    FROM   {0}_tmp t2
    WHERE  t1.index = t2.index
    """.format(table_name, col_name, dtype)
    for col_name, dtype in df.dtypes.map(dtype_mapper).iteritems()
]

with psycopg2.connect(
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"],
    port=5432,
    database=os.environ["PROJECT_NAME"],
) as conn, conn.cursor() as cur:
    for query in queries:
        cur.execute(query)
    conn.commit()
    
    cur.execute("DROP TABLE {}_tmp;".format(table_name))
    conn.commit()


    ALTER TABLE train DROP COLUMN IF EXISTS sex;
    ALTER TABLE train ADD COLUMN sex TEXT;
    UPDATE train t1
    SET    sex = t2.sex
    FROM   train_tmp t2
    WHERE  t1.index = t2.index
    

    ALTER TABLE train DROP COLUMN IF EXISTS sex_int;
    ALTER TABLE train ADD COLUMN sex_int BIGINT;
    UPDATE train t1
    SET    sex_int = t2.sex_int
    FROM   train_tmp t2
    WHERE  t1.index = t2.index
    


In [65]:
from db import table_load
df = table_load("train", cols=["sex"])
df["sex_int"] = df["sex"].map({"male": 0, "female": 1})

[table load (train)] start
[table load (train)] done in 0.0105 s



In [66]:
df

,sex,sex_int
index,,
0,male,0
1,female,1
2,female,1
3,female,1
4,male,0
...,...,...
886,male,0
887,female,1
888,female,1


In [67]:
from db import insert_cols
insert_cols(table_name="train", df=df)